In [1]:
"""Run two below lines to get my_abc module"""
# !git clone https://github.com/thanhttttt/thanh.git
# !pip install -r /content/thanh/requirements.txt
"""Run two below lines to drive"""
# from google.colab import drive
# drive.mount('/content/drive')

'Run two below lines to drive'

# Import

In [2]:
import os
if os.getenv("CUDA_VISIBLE_DEVICES") is None:
    gpu_num = 0 # Use "" to use the CPU
    os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


import sys
sys.path.append('../')
sys.path.append('/content/thanh/')
sys.path.append('../thanh/')
import sionna

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

sionna.config.seed = 42 # Set seed for reproducible random number generation

# Load the required Sionna components
from sionna.nr.my_abc import *

In [3]:
def eval_on_field_data(model, dump_files):
    ls_counter = 0
    nn_counter = 0

    for n, filename in enumerate(dump_files):
        cfg_path = f'{filename}_cfg.txt'
        data_path = f'{filename}_freq.bin'

        caseInfo, sysInfo, ueInfo, _, auxInfo = config_parser(cfg_path)
        freqIQ = data_reader(data_path, (sysInfo['NRxAnt'], 14, sysInfo['BwpNRb']*12))
        ueIQ = tf.constant(freqIQ[None, None, :, :, ueInfo[0]['FirstPrb']*12:(ueInfo[0]['NPrb'] + ueInfo[0]['FirstPrb'])*12])

        sysCfg = SystemConfig(**sysInfo)
        ueCfg = UeConfig(**ueInfo[0])
        myCfg = MyConfig(sysCfg, [ueCfg])
        puschCfg = MyPUSCHConfig(myCfg, slot_number=caseInfo['sf'], frame_number=caseInfo['sfn'])

        simulator = MySimulator(puschCfg)
       
        r = simulator.ref(1)

        """Evaluate ls"""
        h_est, x_hat, llr_det, b_hat, crc_1 = simulator.rec(ueIQ)
        if crc_1:
            ls_counter += 1
            c_rec = simulator.TB_Encoder(b_hat)
        """Evaluate nn"""
        preds = predict(model, ueIQ, r[0:1])
        c_soft = tf.reshape(preds, [preds.shape[0], 1, -1])
        b_hat, crc_2 = simulator.TB_Decoder(c_soft) 

        if crc_2:
            nn_counter += 1
            c_rec = simulator.TB_Encoder(b_hat)
        
        loss = ''
        if crc_1 or crc_2:
            loss = f'({loss_cal(c_soft, c_rec)})'
        # Chỉ lấy tên file (bỏ đường dẫn)
        file_name_only = os.path.basename(filename)

        print(f'{n:3} {file_name_only:<28} | ls: {crc_1 == True} | nn: {crc_2 == True} {loss} | nSnr in flexRan: {auxInfo["nSNR(1)"]:7} | MCS: {puschCfg.tb.mcs_index:2} | NRB: {puschCfg.num_resource_blocks:4}')

    print(len(dump_files), ls_counter, nn_counter)


In [4]:
# Đường dẫn đến thư mục chứa các file config
config_dir = "/workspaces/thanh/data_field/"

filtered_dump_fail_files = []
filtered_dump_pass_files = []

# Duyệt qua các file trong thư mục
for root, dirs, files in os.walk(config_dir):
    for name in files:
        filename = f'{root}/{name}'
        if filename.endswith("_cfg.txt"):  # Chỉ xét các file config
            file_path = os.path.join(config_dir, filename)
            
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()

            DmrsTypeAPos = re.search(r"ue\{1\}\.DmrsTypeAPosition\s*=\s*(\d+)", content)
            DmrsAddPos = re.search(r"ue\{1\}\.DmrsAdditionalPosition\s*=\s*(\d+)", content)
            Mcs = re.search(r"ue\{1\}\.Mcs\s*=\s*(\d+)", content)
            Nrb = re.search(r"ue\{1\}\.NPrb\s*=\s*(\d+)", content)

            if DmrsTypeAPos and DmrsAddPos and Mcs and Nrb:
                DmrsTypeAPos = int(DmrsTypeAPos.group(1))
                DmrsAddPos = int(DmrsAddPos.group(1))
                Mcs = int(Mcs.group(1))
                Nrb = int(Nrb.group(1))

                # Kiểm tra điều kiện
                if DmrsTypeAPos == 3 and DmrsAddPos == 1 and Mcs in {3, 5, 6, 7, 8, 9} and Nrb >= 4:
                    filename = filename[:-len('_cfg.txt')]
                    if 'dump_fail' in filename:
                        filtered_dump_fail_files.append(filename)
                    elif 'dump_pass' in filename:
                        filtered_dump_pass_files.append(filename)

print("Các file thỏa mãn điều kiện:")
print('dump fail: ', len(filtered_dump_fail_files), 'files', '\ndump pass:', len(filtered_dump_pass_files), 'files')

Các file thỏa mãn điều kiện:
dump fail:  52 files 
dump pass: 181 files


In [5]:
_model = CustomNeuralReceiver(training = False)
inputs = tf.zeros([1,48,14,18])
_model(inputs)
load_weights(_model, '../weight_4RB_batchsize_1024_186k_sample_dynamic_config_epoch120.pkl');

Loaded pretrained weights from ../weight_4RB_batchsize_1024_186k_sample_dynamic_config_epoch120.pkl


In [6]:
eval_on_field_data(_model, filtered_dump_pass_files)

I0000 00:00:1742353101.882299   39029 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  0 dump_pass_sfn_496_sf_5       | ls: [[False]] | nn: [[False]]  | nSnr in flexRan:      15 | MCS:  3 | NRB:  140
  1 dump_pass_sfn_1015_sf_4      | ls: [[ True]] | nn: [[ True]] (0.009259739890694618) | nSnr in flexRan:     162 | MCS:  6 | NRB:  140
  2 dump_pass_sfn_1011_sf_5      | ls: [[ True]] | nn: [[ True]] (0.03396828845143318) | nSnr in flexRan:     148 | MCS:  6 | NRB:  140
  3 dump_pass_sfn_950_sf_14      | ls: [[False]] | nn: [[False]]  | nSnr in flexRan: -57     | MCS:  3 | NRB:  136
  4 dump_pass_sfn_862_sf_5       | ls: [[False]] | nn: [[False]]  | nSnr in flexRan: -62     | MCS:  7 | NRB:  140
  5 dump_pass_sfn_870_sf_14      | ls: [[False]] | nn: [[False]]  | nSnr in flexRan: -68     | MCS:  7 | NRB:  136
  6 dump_pass_sfn_951_sf_4       | ls: [[False]] | nn: [[ True]] (0.4771135747432709) | nSnr in flexRan: -52     | MCS:  3 | NRB:  140
  7 dump_pass_sfn_494_sf_15      | ls: [[ True]] | nn: [[ True]] (0.15668092668056488) | nSnr in flexRan:      10 | MCS:  3 | NRB:  